In [61]:
# import libraries
import pandas as pd
import numpy as np 

In [93]:
oil_a = 'mixed_sweet_blend'
oil_b = 'light_smiley'
oil_tbl_a = list(df['mixed_sweet_blend'])
oil_tbl_b = list(df['light_smiley'])


# takes in the names oil_a and oil_b, aswell as their distallation tables, the function would return a new distillation table of the mixture
# append to a list 
def new_mixture_distillation(oil_a,oil_tbl_a,oil_b,oil_tbl_b):
    percent_yeild = [10,20,30,40,50,60,70,80,90,95,99]
    output_df = pd.DataFrame(percent_yeild,columns=['percent_yeild']).set_index('percent_yeild')
    output_df[oil_a] = oil_tbl_a
    output_df[oil_b] = oil_tbl_b
    output_df['mixture_oil'] = (output_df[oil_a] + output_df[oil_b])/2
    output_list = list(output_df['mixture_oil'].round(1))
    while np.isnan(output_list[-1]):
        output_list.pop(-1)
        percent_yeild.pop(-1)
    # print(output_list)
    mymodel = np.poly1d(np.polyfit(percent_yeild,output_list,3))

    print(mymodel)

    # #print a matplotlib graph

    # print(output_df)
    # df['mixture'] = (df['mixed_sweet_blend']+df['light_smiley'])/2 
    
    return "hello"

new_mixture_distillation(oil_a,oil_tbl_a,oil_b,oil_tbl_b)



NameError: name 'percent_yeild1' is not defined

In [1]:
# takes in oil_comp name, oil_tbl, temperature input
def get_maximum_yield(oil_comp,oil_tbl,temp):
    #print a matplotlib graph
    #
    return "hello"

In [2]:
# prints display and rreturns a graph
def display_graph(oil_comp,oil_tbl):
    # prints a matplotlib graph
    return "hello"

SyntaxError: unexpected EOF while parsing (<ipython-input-2-be1735f53907>, line 2)

In [12]:
df = pd.read_csv("sample.csv").set_index('mass_recovered')

print(df)

                mixed_sweet_blend  light_smiley
mass_recovered                                 
10                           80.7          97.7
20                          124.8         158.8
30                          174.9         219.6
40                          232.1         275.7
50                          288.1         328.4
60                          344.8         386.1
70                          407.5         445.5
80                          479.3         521.6
90                          585.8         631.8
95                          654.5         695.5
99                          684.6           NaN


In [66]:
import numpy

x = [1,2,3,5,6,7,8,9,10,12,13,14,15,16,18,19,21,22]
y = [100,90,80,60,60,55,60,65,70,70,75,76,78,79,90,99,99,100]

mymodel = numpy.poly1d(numpy.polyfit(x, y, 3))
print(mymodel)


          3         2
-0.03032 x + 1.343 x - 15.54 x + 113.8
